# Author : Owen Nda Diche


# Feature Engineering (PowerCo Churn)

## Objective
Create additional features that may improve churn prediction.

We will:
1. Remove columns that do not add predictive value (e.g., constant columns)
2. Create new features from existing columns (dates, usage, margins, price metrics)
3. Aggregate historical pricing into customer-level features
4. Merge pricing features with client data
5. Export the final feature set for modelling


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

CLIENT_PATH = "client_data.csv"
PRICE_PATH  = "price_data.csv"

df_client = pd.read_csv(CLIENT_PATH)
df_price  = pd.read_csv(PRICE_PATH)

print("Client shape:", df_client.shape)
print("Price shape :", df_price.shape)

display(df_client.head())
display(df_price.head())


Client shape: (14606, 26)
Price shape : (193002, 8)


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0


,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


## Standardize missing markers + parse dates
We clean common non-standard missing values and convert date columns to datetime.


In [19]:
MISSING_MARKERS = ["MISSING", "missing", "NA", "N/A", "-", "null", "None", ""]

def standardize_missing_markers(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    obj_cols = df.select_dtypes(include=["object"]).columns
    for c in obj_cols:
        df[c] = df[c].replace(MISSING_MARKERS, np.nan)
    return df

def parse_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    date_cols = [c for c in df.columns if ("date" in c.lower()) or ("time" in c.lower())]
    for c in date_cols:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

df_client = standardize_missing_markers(df_client)
df_price  = standardize_missing_markers(df_price)

df_client = parse_date_columns(df_client)
df_price  = parse_date_columns(df_price)

print("Date columns (client):", [c for c in df_client.columns if "date" in c.lower() or "time" in c.lower()])
print("Date columns (price) :", [c for c in df_price.columns if "date" in c.lower() or "time" in c.lower()])


Date columns (client): ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
Date columns (price) : ['price_date']


## Remove columns that add little/no value
- Columns with only **1 unique value**
- Columns with **very high missingness** (threshold configurable)
We keep `id` and `churn` even if rules trigger.


In [20]:
def drop_low_value_columns(df: pd.DataFrame, name: str, missing_thresh: float = 0.90):
    df = df.copy()
    protected = set([c for c in ["id", "churn"] if c in df.columns])

    # constant columns
    nunique = df.nunique(dropna=True)
    constant_cols = [c for c in df.columns if nunique[c] <= 1 and c not in protected]

    # very missing columns
    missing_pct = (df.isna().sum() / len(df))
    mostly_missing_cols = [c for c in df.columns if missing_pct[c] >= missing_thresh and c not in protected]

    drop_cols = sorted(list(set(constant_cols + mostly_missing_cols)))

    print("="*80)
    print(f"{name}: Dropping {len(drop_cols)} low-value columns")
    print("- Constant columns:", constant_cols)
    print(f"- Mostly missing columns (>= {missing_thresh*100:.0f}% missing):", mostly_missing_cols)

    df.drop(columns=drop_cols, inplace=True, errors="ignore")
    return df, drop_cols

df_client, dropped_client = drop_low_value_columns(df_client, "Client Dataset", missing_thresh=0.90)
df_price, dropped_price   = drop_low_value_columns(df_price, "Price Dataset", missing_thresh=0.90)

print("Client shape after drops:", df_client.shape)
print("Price shape after drops :", df_price.shape)


Client Dataset: Dropping 0 low-value columns
- Constant columns: []
- Mostly missing columns (>= 90% missing): []
Price Dataset: Dropping 0 low-value columns
- Constant columns: []
- Mostly missing columns (>= 90% missing): []
Client shape after drops: (14606, 26)
Price shape after drops : (193002, 8)


## Feature engineering: client dataset
We create:
- Date parts (year/month)
- Tenure / recency features (days since activation, days to renewal/end)
- Consumption ratios and totals
- Forecast error features
- Margin / product features


In [21]:
df_client_fe = df_client.copy()

# Choosing a reference date for "days since" features
# Use the most recent date across known date columns (client + price).
ref_candidates = []
for col in ["date_activ", "date_end", "date_modif_prod", "date_renewal"]:
    if col in df_client_fe.columns:
        ref_candidates.append(df_client_fe[col].max())

if "price_date" in df_price.columns:
    ref_candidates.append(df_price["price_date"].max())

REF_DATE = max([d for d in ref_candidates if pd.notna(d)])
print("Reference date used for recency features:", REF_DATE)

#Date features
def add_date_features(df: pd.DataFrame, date_col: str, prefix: str):
    if date_col not in df.columns:
        return df
    df[prefix + "_year"] = df[date_col].dt.year
    df[prefix + "_month"] = df[date_col].dt.month
    return df

for dc, px in [("date_activ", "activ"), ("date_end", "end"), ("date_modif_prod", "modif"), ("date_renewal", "renewal")]:
    df_client_fe = add_date_features(df_client_fe, dc, px)

#Tenure / recency
if "date_activ" in df_client_fe.columns:
    df_client_fe["tenure_days"] = (REF_DATE - df_client_fe["date_activ"]).dt.days

if "date_modif_prod" in df_client_fe.columns:
    df_client_fe["days_since_modif"] = (REF_DATE - df_client_fe["date_modif_prod"]).dt.days

if "date_renewal" in df_client_fe.columns:
    df_client_fe["days_to_renewal"] = (df_client_fe["date_renewal"] - REF_DATE).dt.days

if "date_end" in df_client_fe.columns:
    df_client_fe["days_to_end"] = (df_client_fe["date_end"] - REF_DATE).dt.days

# Helpful flags
if "date_end" in df_client_fe.columns:
    df_client_fe["contract_ended"] = (df_client_fe["date_end"] <= REF_DATE).astype(int)

if "days_to_renewal" in df_client_fe.columns:
    df_client_fe["renewal_within_30d"] = ((df_client_fe["days_to_renewal"] >= 0) & (df_client_fe["days_to_renewal"] <= 30)).astype(int)

# Consumption features
# Total consumption (electric + gas), plus monthly average and ratios
if "cons_12m" in df_client_fe.columns and "cons_gas_12m" in df_client_fe.columns:
    df_client_fe["total_cons_12m"] = df_client_fe["cons_12m"].fillna(0) + df_client_fe["cons_gas_12m"].fillna(0)
else:
    df_client_fe["total_cons_12m"] = df_client_fe.get("cons_12m", 0)

if "cons_12m" in df_client_fe.columns:
    df_client_fe["avg_elec_monthly_cons"] = df_client_fe["cons_12m"] / 12.0

if "total_cons_12m" in df_client_fe.columns:
    denom = df_client_fe["total_cons_12m"].replace(0, np.nan)
    if "cons_gas_12m" in df_client_fe.columns:
        df_client_fe["gas_share_total"] = (df_client_fe["cons_gas_12m"] / denom)

if "cons_last_month" in df_client_fe.columns and "avg_elec_monthly_cons" in df_client_fe.columns:
    denom = df_client_fe["avg_elec_monthly_cons"].replace(0, np.nan)
    df_client_fe["last_month_vs_avg"] = df_client_fe["cons_last_month"] / denom

# --- Forecast vs actual features ---
if "forecast_cons_12m" in df_client_fe.columns and "cons_12m" in df_client_fe.columns:
    df_client_fe["forecast_error_cons_12m"] = df_client_fe["forecast_cons_12m"] - df_client_fe["cons_12m"]
    denom = df_client_fe["cons_12m"].replace(0, np.nan)
    df_client_fe["forecast_ratio_cons_12m"] = df_client_fe["forecast_cons_12m"] / denom

if "forecast_cons_year" in df_client_fe.columns and "cons_12m" in df_client_fe.columns:
    df_client_fe["forecast_error_cons_year"] = df_client_fe["forecast_cons_year"] - df_client_fe["cons_12m"]

# --- Price spread features (forecast prices) ---
if "forecast_price_energy_peak" in df_client_fe.columns and "forecast_price_energy_off_peak" in df_client_fe.columns:
    df_client_fe["forecast_energy_price_spread"] = df_client_fe["forecast_price_energy_peak"] - df_client_fe["forecast_price_energy_off_peak"]

if "forecast_price_pow_off_peak" in df_client_fe.columns:
    # Keep as-is; can be useful directly
    pass

# --- Margin and products ---
if "margin_gross_pow_ele" in df_client_fe.columns and "margin_net_pow_ele" in df_client_fe.columns:
    df_client_fe["gross_minus_net_margin_ele"] = df_client_fe["margin_gross_pow_ele"] - df_client_fe["margin_net_pow_ele"]

if "net_margin" in df_client_fe.columns and "nb_prod_act" in df_client_fe.columns:
    denom = df_client_fe["nb_prod_act"].replace(0, np.nan)
    df_client_fe["net_margin_per_product"] = df_client_fe["net_margin"] / denom

if "pow_max" in df_client_fe.columns and "cons_12m" in df_client_fe.columns:
    denom = df_client_fe["cons_12m"].replace(0, np.nan)
    df_client_fe["powmax_per_elec_cons"] = df_client_fe["pow_max"] / denom

print("Client features created. New shape:", df_client_fe.shape)


Reference date used for recency features: 2017-06-13 00:00:00
Client features created. New shape: (14606, 51)


## Feature engineering: price dataset
The price dataset is transactional/time-based. We create customer-level price features such as:
- Mean and standard deviation (volatility) for each price component
- Latest price values
- Price change over time (latest - earliest, and % change)
- Peak vs off-peak spread
- Last 6 months volatility (optional)


In [22]:
df_price_fe = df_price.copy()

# Ensure price_date exists
assert "price_date" in df_price_fe.columns, "price_date column not found in price_data.csv"
assert "id" in df_price_fe.columns, "id column not found in price_data.csv"

price_cols = [c for c in df_price_fe.columns if c not in ["id", "price_date"]]
print("Price columns found:", price_cols)

# Sort for first/last calculations
df_price_fe = df_price_fe.sort_values(["id", "price_date"])

# --- Aggregations over full history ---
agg_dict = {}
for c in price_cols:
    agg_dict[c] = ["mean", "std", "min", "max"]

price_agg = df_price_fe.groupby("id").agg(agg_dict)
price_agg.columns = ["_".join(col).strip() for col in price_agg.columns.values]
price_agg = price_agg.reset_index()

# --- Latest and earliest rows per customer ---
last_rows = df_price_fe.groupby("id").tail(1).set_index("id")
first_rows = df_price_fe.groupby("id").head(1).set_index("id")

latest_feats = last_rows[price_cols].add_prefix("latest_").reset_index()
earliest_feats = first_rows[price_cols].add_prefix("earliest_").reset_index()

# --- Price change features (latest - earliest) and percent change ---
change_df = pd.DataFrame({"id": last_rows.index})
for c in price_cols:
    latest = last_rows[c]
    earliest = first_rows[c]
    change_df[f"chg_{c}"] = (latest - earliest).values
    change_df[f"pct_chg_{c}"] = ((latest - earliest) / earliest.replace(0, np.nan)).values

# --- Spread features (peak - off_peak) for var and fix where present ---
def safe_spread(df, a, b, name):
    if a in df.columns and b in df.columns:
        df[name] = df[a] - df[b]
    return df

# Create spread features on latest prices (most relevant for churn decision)
latest_spreads = latest_feats.copy()
latest_spreads = safe_spread(latest_spreads, "latest_price_peak_var", "latest_price_off_peak_var", "latest_spread_peak_offpeak_var")
latest_spreads = safe_spread(latest_spreads, "latest_price_mid_peak_var", "latest_price_off_peak_var", "latest_spread_mid_offpeak_var")
latest_spreads = safe_spread(latest_spreads, "latest_price_peak_fix", "latest_price_off_peak_fix", "latest_spread_peak_offpeak_fix")
latest_spreads = safe_spread(latest_spreads, "latest_price_mid_peak_fix", "latest_price_off_peak_fix", "latest_spread_mid_offpeak_fix")

# --- OPTIONAL: last 6 months volatility (uses REF_DATE defined earlier) ---
six_months_cut = REF_DATE - pd.Timedelta(days=183)
df_last6 = df_price_fe[df_price_fe["price_date"] >= six_months_cut].copy()

agg6_dict = {c: ["mean", "std"] for c in price_cols}
price_6m = df_last6.groupby("id").agg(agg6_dict)
price_6m.columns = [f"6m_{a}_{b}" for (a, b) in price_6m.columns]
price_6m = price_6m.reset_index()

# --- Combine all price features ---
price_features = price_agg.merge(latest_feats, on="id", how="left") \
                          .merge(earliest_feats, on="id", how="left") \
                          .merge(change_df, on="id", how="left") \
                          .merge(latest_spreads[["id"] + [c for c in latest_spreads.columns if c.startswith("latest_spread_")]], on="id", how="left") \
                          .merge(price_6m, on="id", how="left")

print("Price features shape:", price_features.shape)
display(price_features.head())


Price columns found: ['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']
Price features shape: (16096, 65)


,id,price_off_peak_var_mean,price_off_peak_var_std,price_off_peak_var_min,price_off_peak_var_max,price_peak_var_mean,price_peak_var_std,price_peak_var_min,price_peak_var_max,price_mid_peak_var_mean,price_mid_peak_var_std,price_mid_peak_var_min,price_mid_peak_var_max,price_off_peak_fix_mean,price_off_peak_fix_std,price_off_peak_fix_min,price_off_peak_fix_max,price_peak_fix_mean,price_peak_fix_std,price_peak_fix_min,price_peak_fix_max,price_mid_peak_fix_mean,price_mid_peak_fix_std,price_mid_peak_fix_min,price_mid_peak_fix_max,latest_price_off_peak_var,latest_price_peak_var,latest_price_mid_peak_var,latest_price_off_peak_fix,latest_price_peak_fix,latest_price_mid_peak_fix,earliest_price_off_peak_var,earliest_price_peak_var,earliest_price_mid_peak_var,earliest_price_off_peak_fix,earliest_price_peak_fix,earliest_price_mid_peak_fix,chg_price_off_peak_var,pct_chg_price_off_peak_var,chg_price_peak_var,pct_chg_price_peak_var,chg_price_mid_peak_var,pct_chg_price_mid_peak_var,chg_price_off_peak_fix,pct_chg_price_off_peak_fix,chg_price_peak_fix,pct_chg_price_peak_fix,chg_price_mid_peak_fix,pct_chg_price_mid_peak_fix,latest_spread_peak_offpeak_var,latest_spread_mid_offpeak_var,latest_spread_peak_offpeak_fix,latest_spread_mid_offpeak_fix,6m_price_off_peak_var_mean,6m_price_off_peak_var_std,6m_price_peak_var_mean,6m_price_peak_var_std,6m_price_mid_peak_var_mean,6m_price_mid_peak_var_std,6m_price_off_peak_fix_mean,6m_price_off_peak_fix_std,6m_price_peak_fix_mean,6m_price_peak_fix_std,6m_price_mid_peak_fix_mean,6m_price_mid_peak_fix_std
0,0002203ffbb812588b632b9e628cc38d,0.124338,0.003976,0.119906,0.128067,0.103794,0.001989,0.101673,0.105842,0.073160,0.001368,0.070232,0.073773,40.701732,6.341481e-02,40.565969,40.728885,24.421038,0.038049,24.339581,24.43733,16.280694,0.025366,16.226389,16.291555,0.119906,0.101673,0.073719,40.728885,24.43733,16.291555,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389,-0.006192,-0.049105,-0.002302,-0.022140,0.003487,0.049650,0.162916,4.016066e-03,0.097749,0.004016,0.065166,0.004016,-0.018233,-0.046187,-16.291555,-24.43733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0004351ebdd665e6ee664792efc4fd13,0.146426,0.002197,0.143943,0.148405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.385450,8.753223e-02,44.266931,44.444710,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.143943,0.000000,0.000000,44.444710,0.00000,0.000000,0.148047,0.000000,0.000000,44.266931,0.000000,0.000000,-0.004104,-0.027721,0.000000,NaN,0.000000,NaN,0.177779,4.016063e-03,0.000000,NaN,0.000000,NaN,-0.143943,-0.143943,-44.444710,-44.44471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.026008,0.150837,0.205742,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.319710,7.723930e-01,44.444710,45.944710,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.201280,0.000000,0.000000,45.944710,0.00000,0.000000,0.150837,0.000000,0.000000,44.444710,0.000000,0.000000,0.050443,0.334421,0.000000,NaN,0.000000,NaN,1.500000,3.374980e-02,0.000000,NaN,0.000000,NaN,-0.201280,-0.201280,-45.944710,-45.94471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0010ee3855fdea87602a5b7aba8e42de,0.118757,0.005049,0.113068,0.123086,0.098292,0.002580,0.095385,0.100505,0.069032,0.000403,0.068646,0.069463,40.647427,8.507958e-02,40.565969,40.728885,24.388455,0.051048,24.339578,24.43733,16.258971,0.034032,16.226383,16.291555,0.113068,0.095385,0.069409,40.728885,24.43733,16.291555,0.123086,0.100505,0.068646,40.565969,24.339581,16.226389,-0.010018,-0.081390,-0.005120,-0.050943,0.000763,0.011115,0.162916,4.016066e-03,0.097749,0.004016,0.065166,0.004016,-0.017683,-0.043659,-16.291555,-24.43733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00114d74e963e47177db89bc70108537,0.147926,0.002202,0.145440,0.149902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.266930,5.908392e-07,44.266930,44.266931,0.000000,0

## Merge datasets
Merge price features into the client dataset using the shared key: `id`.


In [23]:
assert "id" in df_client_fe.columns, "id not found in client dataset"

df_merged = df_client_fe.merge(price_features, on="id", how="left")

print("Merged dataset shape:", df_merged.shape)

# Quick check of missing price features after merge
merged_price_missing = df_merged[[c for c in price_features.columns if c != "id"]].isna().mean().sort_values(ascending=False).head(10)
print("\nTop 10 merged price feature missing rates:")
display(merged_price_missing)


Merged dataset shape: (14606, 115)

Top 10 merged price feature missing rates:


6m_price_peak_fix_mean        1.0
6m_price_peak_fix_std         1.0
6m_price_mid_peak_fix_mean    1.0
6m_price_mid_peak_fix_std     1.0
6m_price_peak_var_std         1.0
6m_price_peak_var_mean        1.0
6m_price_off_peak_var_std     1.0
6m_price_off_peak_var_mean    1.0
6m_price_off_peak_fix_std     1.0
6m_price_off_peak_fix_mean    1.0
dtype: float64

## Final cleanup
- Drop raw datetime columns (models can't directly use datetime)
- Fill NaNs for engineered numeric fields
We keep `id` and `churn`.


In [24]:
df_final = df_merged.copy()

# Drop datetime columns (keep the engineered year/month and day-diff features instead)
datetime_cols = df_final.select_dtypes(include=["datetime64[ns]"]).columns.tolist()
df_final.drop(columns=datetime_cols, inplace=True, errors="ignore")
print("Dropped datetime columns:", datetime_cols)

# fill NaNs in numeric columns with median (common baseline)
num_cols = df_final.select_dtypes(include=[np.number]).columns
for c in num_cols:
    if c not in ["churn"]:  # leave churn untouched
        df_final[c] = df_final[c].fillna(df_final[c].median())

print("Final dataset shape:", df_final.shape)
display(df_final.head())


Dropped datetime columns: ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
Final dataset shape: (14606, 111)


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn,activ_year,activ_month,end_year,end_month,modif_year,modif_month,renewal_year,renewal_month,tenure_days,days_since_modif,days_to_renewal,days_to_end,contract_ended,renewal_within_30d,total_cons_12m,avg_elec_monthly_cons,gas_share_total,last_month_vs_avg,forecast_error_cons_12m,forecast_ratio_cons_12m,forecast_error_cons_year,forecast_energy_price_spread,gross_minus_net_margin_ele,net_margin_per_product,powmax_per_elec_cons,price_off_peak_var_mean,price_off_peak_var_std,price_off_peak_var_min,price_off_peak_var_max,price_peak_var_mean,price_peak_var_std,price_peak_var_min,price_peak_var_max,price_mid_peak_var_mean,price_mid_peak_var_std,price_mid_peak_var_min,price_mid_peak_var_max,price_off_peak_fix_mean,price_off_peak_fix_std,price_off_peak_fix_min,price_off_peak_fix_max,price_peak_fix_mean,price_peak_fix_std,price_peak_fix_min,price_peak_fix_max,price_mid_peak_fix_mean,price_mid_peak_fix_std,price_mid_peak_fix_min,price_mid_peak_fix_max,latest_price_off_peak_var,latest_price_peak_var,latest_price_mid_peak_var,latest_price_off_peak_fix,latest_price_peak_fix,latest_price_mid_peak_fix,earliest_price_off_peak_var,earliest_price_peak_var,earliest_price_mid_peak_var,earliest_price_off_peak_fix,earliest_price_peak_fix,earliest_price_mid_peak_fix,chg_price_off_peak_var,pct_chg_price_off_peak_var,chg_price_peak_var,pct_chg_price_peak_var,chg_price_mid_peak_var,pct_chg_price_mid_peak_var,chg_price_off_peak_fix,pct_chg_price_off_peak_fix,chg_price_peak_fix,pct_chg_price_peak_fix,chg_price_mid_peak_fix,pct_chg_price_mid_peak_fix,latest_spread_peak_offpeak_var,latest_spread_mid_offpeak_var,latest_spread_peak_offpeak_fix,latest_spread_mid_offpeak_fix,6m_price_off_peak_var_mean,6m_price_off_peak_var_std,6m_price_peak_var_mean,6m_price_peak_var_std,6m_price_mid_peak_var_mean,6m_price_mid_peak_var_std,6m_price_off_peak_fix_mean,6m_price_off_peak_fix_std,6m_price_peak_fix_mean,6m_price_peak_fix_std,6m_price_mid_peak_fix_mean,6m_price_mid_peak_fix_std
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,2013,6,2016,6,2015,11,2015,6,1459,590,-721,-363,1,0,54946,0.000000,1.0,0.873417,0.00,0.102448,0,-0.016339,0.0,339.495,0.001106,0.124787,0.007829,0.117479,0.146033,0.100749,0.005126,0.085483,0.103963,0.066530,0.020983,0.000000,0.073873,40.942265,1.050136,40.565969,44.266930,22.352010,7.039226,0.000000,24.43733,14.901340,4.692817,0.000000,16.291555,0.146033,0.085483,0.000000,44.266930,0.00000,0.000000,0.125976,0.103395,0.071536,40.565969,24.339581,16.226389,0.020057,0.159213,-0.017912,-0.173239,-0.071536,-1.000000,3.700961,0.091233,-24.339581,-1.000000,-16.226389,-1.000000,-0.060550,-0.146033,-44.266930,-44.26693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d29c2c54acc38ff3c0614d0a653813dd,NaN,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,2009,8,2016,8,2009,8,2015,8,2853,2853,-652,-287,1,0,4660,388.333333,0.0,0.000000,-4470.05,0.040762,-4660,-0.145711,0.0,18.890,0.002961,0.149609,0.002212,0.146033,0.151367,0.007124,0.024677,0.000000,0.085483,0.000000,0.000000,0.000000,0.000000,44.311375,0.080404,44.266930,44.444710,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.147600,0.000000,0.000000,44.444710,0.00000,0.000000,0.151367,0.000000,0.000000,44.266931,0.000000,0.000000,-0.003767,-0.024887,0.000000,-0.028086,0.000000,0.028527,0.177779,0.004016,0.000000,0.004016,0.000000,0.004016,-0.147600,-0.147600,-44.444710,-44.44471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Export outputs
1) Save the feature engineered dataset
2) Write a standalone `feature_engineering.py` script (upload this if required)


In [15]:
OUTPUT_CSV = "data_feature_engineered.csv"
df_final.to_csv(OUTPUT_CSV, index=False)
print("Saved:", OUTPUT_CSV)


Saved: data_feature_engineered.csv


In [16]:
# A Python file containing the full pipeline (ready to upload)
PY_FILE = "feature_engineering.py"

script = r'''
import pandas as pd
import numpy as np

CLIENT_PATH = "client_data.csv"
PRICE_PATH  = "price_data.csv"
OUTPUT_CSV  = "data_feature_engineered.csv"

MISSING_MARKERS = ["MISSING", "missing", "NA", "N/A", "-", "null", "None", ""]

def standardize_missing_markers(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    obj_cols = df.select_dtypes(include=["object"]).columns
    for c in obj_cols:
        df[c] = df[c].replace(MISSING_MARKERS, np.nan)
    return df

def parse_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    date_cols = [c for c in df.columns if ("date" in c.lower()) or ("time" in c.lower())]
    for c in date_cols:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

def drop_low_value_columns(df: pd.DataFrame, missing_thresh: float = 0.90):
    df = df.copy()
    protected = set([c for c in ["id", "churn"] if c in df.columns])

    nunique = df.nunique(dropna=True)
    constant_cols = [c for c in df.columns if nunique[c] <= 1 and c not in protected]

    missing_pct = (df.isna().sum() / len(df))
    mostly_missing_cols = [c for c in df.columns if missing_pct[c] >= missing_thresh and c not in protected]

    drop_cols = sorted(list(set(constant_cols + mostly_missing_cols)))
    df.drop(columns=drop_cols, inplace=True, errors="ignore")
    return df

def add_date_features(df: pd.DataFrame, date_col: str, prefix: str):
    if date_col not in df.columns:
        return df
    df[prefix + "_year"] = df[date_col].dt.year
    df[prefix + "_month"] = df[date_col].dt.month
    return df

def main():
    df_client = pd.read_csv(CLIENT_PATH)
    df_price  = pd.read_csv(PRICE_PATH)

    df_client = standardize_missing_markers(df_client)
    df_price  = standardize_missing_markers(df_price)

    df_client = parse_date_columns(df_client)
    df_price  = parse_date_columns(df_price)

    df_client = drop_low_value_columns(df_client, missing_thresh=0.90)
    df_price  = drop_low_value_columns(df_price, missing_thresh=0.90)

    # reference date
    ref_candidates = []
    for col in ["date_activ", "date_end", "date_modif_prod", "date_renewal"]:
        if col in df_client.columns:
            ref_candidates.append(df_client[col].max())
    if "price_date" in df_price.columns:
        ref_candidates.append(df_price["price_date"].max())
    REF_DATE = max([d for d in ref_candidates if pd.notna(d)])

    # client FE
    df_client_fe = df_client.copy()
    for dc, px in [("date_activ","activ"), ("date_end","end"), ("date_modif_prod","modif"), ("date_renewal","renewal")]:
        df_client_fe = add_date_features(df_client_fe, dc, px)

    if "date_activ" in df_client_fe.columns:
        df_client_fe["tenure_days"] = (REF_DATE - df_client_fe["date_activ"]).dt.days
    if "date_modif_prod" in df_client_fe.columns:
        df_client_fe["days_since_modif"] = (REF_DATE - df_client_fe["date_modif_prod"]).dt.days
    if "date_renewal" in df_client_fe.columns:
        df_client_fe["days_to_renewal"] = (df_client_fe["date_renewal"] - REF_DATE).dt.days
        df_client_fe["renewal_within_30d"] = ((df_client_fe["days_to_renewal"] >= 0) & (df_client_fe["days_to_renewal"] <= 30)).astype(int)
    if "date_end" in df_client_fe.columns:
        df_client_fe["days_to_end"] = (df_client_fe["date_end"] - REF_DATE).dt.days
        df_client_fe["contract_ended"] = (df_client_fe["date_end"] <= REF_DATE).astype(int)

    if "cons_12m" in df_client_fe.columns and "cons_gas_12m" in df_client_fe.columns:
        df_client_fe["total_cons_12m"] = df_client_fe["cons_12m"].fillna(0) + df_client_fe["cons_gas_12m"].fillna(0)
    else:
        df_client_fe["total_cons_12m"] = df_client_fe.get("cons_12m", 0)

    if "cons_12m" in df_client_fe.columns:
        df_client_fe["avg_elec_monthly_cons"] = df_client_fe["cons_12m"] / 12.0

    if "total_cons_12m" in df_client_fe.columns and "cons_gas_12m" in df_client_fe.columns:
        denom = df_client_fe["total_cons_12m"].replace(0, np.nan)
        df_client_fe["gas_share_total"] = df_client_fe["cons_gas_12m"] / denom

    if "cons_last_month" in df_client_fe.columns and "avg_elec_monthly_cons" in df_client_fe.columns:
        denom = df_client_fe["avg_elec_monthly_cons"].replace(0, np.nan)
        df_client_fe["last_month_vs_avg"] = df_client_fe["cons_last_month"] / denom

    if "forecast_cons_12m" in df_client_fe.columns and "cons_12m" in df_client_fe.columns:
        df_client_fe["forecast_error_cons_12m"] = df_client_fe["forecast_cons_12m"] - df_client_fe["cons_12m"]
        denom = df_client_fe["cons_12m"].replace(0, np.nan)
        df_client_fe["forecast_ratio_cons_12m"] = df_client_fe["forecast_cons_12m"] / denom

    if "forecast_price_energy_peak" in df_client_fe.columns and "forecast_price_energy_off_peak" in df_client_fe.columns:
        df_client_fe["forecast_energy_price_spread"] = df_client_fe["forecast_price_energy_peak"] - df_client_fe["forecast_price_energy_off_peak"]

    if "margin_gross_pow_ele" in df_client_fe.columns and "margin_net_pow_ele" in df_client_fe.columns:
        df_client_fe["gross_minus_net_margin_ele"] = df_client_fe["margin_gross_pow_ele"] - df_client_fe["margin_net_pow_ele"]

    if "net_margin" in df_client_fe.columns and "nb_prod_act" in df_client_fe.columns:
        denom = df_client_fe["nb_prod_act"].replace(0, np.nan)
        df_client_fe["net_margin_per_product"] = df_client_fe["net_margin"] / denom

    if "pow_max" in df_client_fe.columns and "cons_12m" in df_client_fe.columns:
        denom = df_client_fe["cons_12m"].replace(0, np.nan)
        df_client_fe["powmax_per_elec_cons"] = df_client_fe["pow_max"] / denom

    # price FE
    df_price_fe = df_price.copy()
    df_price_fe = df_price_fe.sort_values(["id", "price_date"])
    price_cols = [c for c in df_price_fe.columns if c not in ["id","price_date"]]

    agg_dict = {c: ["mean","std","min","max"] for c in price_cols}
    price_agg = df_price_fe.groupby("id").agg(agg_dict)
    price_agg.columns = ["_".join(col) for col in price_agg.columns.values]
    price_agg = price_agg.reset_index()

    last_rows = df_price_fe.groupby("id").tail(1).set_index("id")
    first_rows = df_price_fe.groupby("id").head(1).set_index("id")

    latest_feats = last_rows[price_cols].add_prefix("latest_").reset_index()
    earliest_feats = first_rows[price_cols].add_prefix("earliest_").reset_index()

    change_df = pd.DataFrame({"id": last_rows.index})
    for c in price_cols:
        latest = last_rows[c]
        earliest = first_rows[c]
        change_df[f"chg_{c}"] = (latest - earliest).values
        change_df[f"pct_chg_{c}"] = ((latest - earliest) / earliest.replace(0, np.nan)).values

    # spreads on latest
    latest_spreads = latest_feats.copy()
    if "latest_price_peak_var" in latest_spreads.columns and "latest_price_off_peak_var" in latest_spreads.columns:
        latest_spreads["latest_spread_peak_offpeak_var"] = latest_spreads["latest_price_peak_var"] - latest_spreads["latest_price_off_peak_var"]
    if "latest_price_mid_peak_var" in latest_spreads.columns and "latest_price_off_peak_var" in latest_spreads.columns:
        latest_spreads["latest_spread_mid_offpeak_var"] = latest_spreads["latest_price_mid_peak_var"] - latest_spreads["latest_price_off_peak_var"]
    if "latest_price_peak_fix" in latest_spreads.columns and "latest_price_off_peak_fix" in latest_spreads.columns:
        latest_spreads["latest_spread_peak_offpeak_fix"] = latest_spreads["latest_price_peak_fix"] - latest_spreads["latest_price_off_peak_fix"]
    if "latest_price_mid_peak_fix" in latest_spreads.columns and "latest_price_off_peak_fix" in latest_spreads.columns:
        latest_spreads["latest_spread_mid_offpeak_fix"] = latest_spreads["latest_price_mid_peak_fix"] - latest_spreads["latest_price_off_peak_fix"]

    six_months_cut = REF_DATE - pd.Timedelta(days=183)
    df_last6 = df_price_fe[df_price_fe["price_date"] >= six_months_cut].copy()
    agg6_dict = {c: ["mean","std"] for c in price_cols}
    price_6m = df_last6.groupby("id").agg(agg6_dict)
    price_6m.columns = [f"6m_{a}_{b}" for (a, b) in price_6m.columns]
    price_6m = price_6m.reset_index()

    price_features = price_agg.merge(latest_feats, on="id", how="left") \
                              .merge(earliest_feats, on="id", how="left") \
                              .merge(change_df, on="id", how="left") \
                              .merge(latest_spreads[["id"] + [c for c in latest_spreads.columns if c.startswith("latest_spread_")]], on="id", how="left") \
                              .merge(price_6m, on="id", how="left")

    df_merged = df_client_fe.merge(price_features, on="id", how="left")

    # drop datetime cols
    datetime_cols = df_merged.select_dtypes(include=["datetime64[ns]"]).columns.tolist()
    df_final = df_merged.drop(columns=datetime_cols, errors="ignore")

    # fill numeric NaNs
    num_cols = df_final.select_dtypes(include=[np.number]).columns
    for c in num_cols:
        if c != "churn":
            df_final[c] = df_final[c].fillna(df_final[c].median())

    df_final.to_csv(OUTPUT_CSV, index=False)
    print("Saved:", OUTPUT_CSV)

if __name__ == "__main__":
    main()
'''

with open(PY_FILE, "w", encoding="utf-8") as f:
    f.write(script)

print("Saved:", PY_FILE)


Saved: feature_engineering.py
